# Age-structured ICU demand sampler

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy.optimize import minimize 
from datetime import timedelta
import datetime as dt
from IPython.display import Image
import requests

In [42]:
name = 'São Carlos'
state_or_city = 'city'
pop0 = 	251983
p_SUS = 0.6278

state = 'SP'
name_file = 'Sao_Carlos'

## Data

### Brazil

#### COVID-19 DATA

Data source: [Brasil.IO](https://brasil.io/dataset/covid19/caso). Observe the database changes every day.



In [43]:
#url = "https://brasil.io/dataset/covid19/caso/?format=csv"
#filename = 'data/brazil_' + url.split("/")[-3] + '.csv'
#with open(filename, "wb") as f:
#    r = requests.get(url)
#    f.write(r.content)

To keep the same analysis, we keep the data basis from June 11.

In [44]:
filename = 'data/brazil_covid19_06_11.csv'

df = pd.read_csv(filename)
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-06-10,AC,Acrelândia,city,167,2,True,15256.0,1200013.0,1094.65128,0.0120
1,2020-06-10,AC,Assis Brasil,city,79,4,True,7417.0,1200054.0,1065.12067,0.0506
2,2020-06-10,AC,Brasiléia,city,176,5,True,26278.0,1200104.0,669.76178,0.0284
3,2020-06-10,AC,Bujari,city,90,1,True,10266.0,1200138.0,876.68030,0.0111
4,2020-06-10,AC,Capixaba,city,73,2,True,11733.0,1200179.0,622.17677,0.0274


Select a particular state or city:

In [45]:
df = df[ df['place_type'] == state_or_city]
df = df[ df[state_or_city] == name ]

df_I = df.groupby('date')[['confirmed']].sum()
df_I.index = pd.to_datetime(df_I.index)
today = df_I.index[-1] + timedelta(days=1)

Last day of data used to run this notebook:

In [46]:
fit_until = df_I.index[-1].strftime('%m-%d')
fit_until

'06-10'

#### Population data combined with ICU adimission probability by age

Data source: [IBGE](https://www.ibge.gov.br/apps/populacao/projecao/).

In [47]:
# State of São Paulo
file = 'data/pop_age_str_IBGE_2020_' + state + '.csv'

# Other states
# file = 'data/pop_age_str_IBGE_2020_' + name + '.csv'

df_age = pd.read_csv(file)
df_age.loc[0, 'Age'] = '00-04'
df_age.loc[1, 'Age'] = '05-09'
df_age['AGE_prob'] = df_age['Total'] / df_age['Total'].sum()

# Selected state
#pop0 = df_age['Total'].sum().item()

In [48]:
df_age_ICU = pd.DataFrame(columns=['Age', 'ICU_prob'])
df_age_ICU['Age'] = ['0-19', '20-44', '45-54', '55-64', '65-74', '75-84', '85+']
df_age_ICU['ICU_prob'] = [0, 4.2, 10.4, 11.2, 18.8, 31, 29]

In [49]:
ICU_prob = [0., 0., 0., 0., 0.042, 
            0.042, 0.042, 0.042, 0.042, 0.104,
            0.104, 0.112, 0.112, 0.188, 0.188,
            0.31, 0.31, 0.29, 0.29]

df_age['ICU_prob'] = ICU_prob

## Sampling from age-structed population probability and ICU admission probability

Function to be used to performe both sampling over time.

In [50]:
def ICU_samp(df, n, n_samp_AGE_max= 1000, n_samp_AGE_min= 100, n_samp_ICU_max= 1000, n_samp_ICU_min= 100):

    df_samp = pd.DataFrame(columns= df['Age'])

    for j in range(n_samp_AGE_max):
    
        samp = np.random.choice(df['Age'], 
                                n, 
                                p= list(df['AGE_prob']) )

        unique, counts = np.unique(samp, return_counts= True)
    
        for l in range(len(unique)):
            df_samp.loc[j, unique[l]] = counts[l]


    df_samp = df_samp.fillna(0)
    df['n_mean'] = list(df_samp.mean(axis= 0))
    df['n_std']  = list(df_samp.std(axis =0))

    df = df.set_index('Age')

    for age in df.index:
    
        aux_ = []

        for j in range(n_samp_ICU_max):      
    
            samp = np.random.uniform(size= int(df.loc[age]['n_mean']))  
            samp_ICU = samp < df.loc[age]['ICU_prob']
            aux_.append(samp_ICU.sum())


        df.loc[age, 'n_mean_ICU']  = np.mean(aux_)
        df.loc[age, 'n_std_ICU']   = np.std(aux_)
        
    df['n_std_ICU'] = np.sqrt( df['n_std']**2 +  df['n_std_ICU']**2)
    
    return df

Running the function over both scenarios:

In [51]:
df_I

,confirmed
date,
2020-04-06,1
2020-04-07,1
2020-04-08,1
2020-04-09,2
2020-04-10,4
...,...
2020-06-06,224
2020-06-07,228
2020-06-08,226


In [36]:
df1_ = []

for j in range(len(df_I)):
    
    df1 = ICU_samp(df= df_age.reset_index(), 
                   n= df_I.iloc[j][0], 
                   n_samp_AGE_max= 100, n_samp_AGE_min= 100,
                   n_samp_ICU_max= 100, n_samp_ICU_min= 100)

    
    df1_.append(df1)

In [37]:
df1_[0]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU
Age,,,,,,,,
00-04,0,3038361,0.065638,0.000,0.07,0.256432,0.0,0.256432
05-09,1,3042098,0.065719,0.000,0.08,0.272660,0.0,0.272660
10-14,2,2957679,0.063895,0.000,0.04,0.196946,0.0,0.196946
15-19,3,3127121,0.067556,0.000,0.07,0.256432,0.0,0.256432
20-24,4,3448513,0.074499,0.042,0.07,0.256432,0.0,0.256432
25-29,5,3511090,0.075851,0.042,0.10,0.301511,0.0,0.301511
30-34,6,3809963,0.082308,0.042,0.10,0.301511,0.0,0.301511
35-39,7,3854180,0.083263,0.042,0.07,0.256432,0.0,0.256432
40-44,8,3548754,0.076665,0.042,0.05,0.219043,0.0,0.219043


### Taking into account removal from ICU after `T_ICU` days

In [20]:
def correction(x, df_, T_ICU= 14):

    df_[x]['n_mean_ICU_cor'] = 0.
    df_[x]['n_std_ICU_cor'] = 0.
    
    if x <= T_ICU:
                   
        df_[x]['n_mean_ICU_cor'] = df_[x]['n_mean_ICU']
        df_[x]['n_std_ICU_cor']   = df_[x]['n_std_ICU']
             
    else:
        
        delta = df_[x]['n_mean_ICU'] - df_[x - T_ICU]['n_mean_ICU']
        
        df_[x]['n_mean_ICU_cor'] = np.heaviside(delta, 0) * delta
        df_[x]['n_std_ICU_cor']  = np.sqrt(df_[x]['n_std_ICU']**2 + df_[x - T_ICU]['n_std_ICU']**2)

In [25]:
df1_[40]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU,n_mean_ICU_cor,n_std_ICU_cor
Age,,,,,,,,,,
00-04,0,3038361,0.065638,0.000,4.59,1.969951,0.00,1.969951,0.00,2.196301
05-09,1,3042098,0.065719,0.000,4.17,2.040276,0.00,2.040276,0.00,2.234486
10-14,2,2957679,0.063895,0.000,4.22,1.982652,0.00,1.982652,0.00,2.203762
15-19,3,3127121,0.067556,0.000,4.45,1.961112,0.00,1.961112,0.00,2.186968
20-24,4,3448513,0.074499,0.042,4.66,1.902789,0.15,1.941161,0.14,2.129691
25-29,5,3511090,0.075851,0.042,5.11,2.159756,0.21,2.206909,0.21,2.431972
30-34,6,3809963,0.082308,0.042,5.94,2.255498,0.19,2.293725,0.14,2.487278
35-39,7,3854180,0.083263,0.042,5.09,2.256730,0.24,2.301136,0.19,2.568595
40-44,8,3548754,0.076665,0.042,5.03,2.185605,0.23,2.234719,0.17,2.436313


In [23]:
T_ICU = 14

for x in range(len(df1_)):
    
    correction(x, df_= df1_, T_ICU= T_ICU)

In [40]:
df1_[45]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU
Age,,,,,,,,
00-04,0,3038361,0.065638,0.000,5.59,2.554833,0.00,2.554833
05-09,1,3042098,0.065719,0.000,5.42,2.175065,0.00,2.175065
10-14,2,2957679,0.063895,0.000,5.36,2.426787,0.00,2.426787
15-19,3,3127121,0.067556,0.000,5.71,2.166294,0.00,2.166294
20-24,4,3448513,0.074499,0.042,5.92,2.218745,0.16,2.257704
25-29,5,3511090,0.075851,0.042,6.22,2.451880,0.27,2.499763
30-34,6,3809963,0.082308,0.042,7.05,2.426183,0.29,2.492441
35-39,7,3854180,0.083263,0.042,7.45,2.442780,0.30,2.509417
40-44,8,3548754,0.076665,0.042,6.63,2.158633,0.27,2.212871


## Collecting daily averages

Given a list `df_` of dataframes, the function `daily_av` collect averages of collumns values over a `timeseries_data` period. The return is a daaframe called `df_ICU`.

In [ ]:
for 

In [52]:
df1_[0]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU
Age,,,,,,,,
00-04,0,3038361,0.065638,0.000,0.07,0.256432,0.0,0.256432
05-09,1,3042098,0.065719,0.000,0.08,0.272660,0.0,0.272660
10-14,2,2957679,0.063895,0.000,0.04,0.196946,0.0,0.196946
15-19,3,3127121,0.067556,0.000,0.07,0.256432,0.0,0.256432
20-24,4,3448513,0.074499,0.042,0.07,0.256432,0.0,0.256432
25-29,5,3511090,0.075851,0.042,0.10,0.301511,0.0,0.301511
30-34,6,3809963,0.082308,0.042,0.10,0.301511,0.0,0.301511
35-39,7,3854180,0.083263,0.042,0.07,0.256432,0.0,0.256432
40-44,8,3548754,0.076665,0.042,0.05,0.219043,0.0,0.219043


In [53]:
df1_[0]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU
Age,,,,,,,,
00-04,0,3038361,0.065638,0.000,0.07,0.256432,0.0,0.256432
05-09,1,3042098,0.065719,0.000,0.08,0.272660,0.0,0.272660
10-14,2,2957679,0.063895,0.000,0.04,0.196946,0.0,0.196946
15-19,3,3127121,0.067556,0.000,0.07,0.256432,0.0,0.256432
20-24,4,3448513,0.074499,0.042,0.07,0.256432,0.0,0.256432
25-29,5,3511090,0.075851,0.042,0.10,0.301511,0.0,0.301511
30-34,6,3809963,0.082308,0.042,0.10,0.301511,0.0,0.301511
35-39,7,3854180,0.083263,0.042,0.07,0.256432,0.0,0.256432
40-44,8,3548754,0.076665,0.042,0.05,0.219043,0.0,0.219043


In [72]:
df1_[12]

,index,Total,AGE_prob,ICU_prob,n_mean,n_std,n_mean_ICU,n_std_ICU
Age,,,,,,,,
00-04,0,3038361,0.065638,0.000,0.35,0.609272,0.0,0.609272
05-09,1,3042098,0.065719,0.000,0.47,0.626921,0.0,0.626921
10-14,2,2957679,0.063895,0.000,0.45,0.672324,0.0,0.672324
15-19,3,3127121,0.067556,0.000,0.54,0.730573,0.0,0.730573
20-24,4,3448513,0.074499,0.042,0.41,0.621094,0.0,0.621094
25-29,5,3511090,0.075851,0.042,0.46,0.626357,0.0,0.626357
30-34,6,3809963,0.082308,0.042,0.67,0.766139,0.0,0.766139
35-39,7,3854180,0.083263,0.042,0.63,0.836720,0.0,0.836720
40-44,8,3548754,0.076665,0.042,0.53,0.717107,0.0,0.717107


In [55]:
def daily_av(df_, timeseries_data, date, SUS= True, p_SUS= 0.6278, n_samp_max= 1000, n_samp_min= 100):

    #n_mean_    = []
    #n_std_     = []
    #n_mean_ICU_ = []
    #n_std_ICU_  = []

    ##for j in range(len(df_)):

    ##    n_mean_.append(df_[j]['n_mean'].sum())
    ##    n_std_.append( np.sqrt((df_[j]['n_std']**2).sum()) )
        
    ##    n_mean_ICU_.append(df_[j]['n_mean_ICU'].sum())
    ##  n_std_ICU_.append( np.sqrt((df_[j]['n_std_ICU']**2).sum()) )

    names = ['date', 'n_mean', 'n_std', 'n_mean_ICU', 'n_std_ICU']
    df_ICU = pd.DataFrame(columns= names)
    
    #df_ICU['date'] = timeseries_data
    df_ICU['date'] = date

    
    df_ICU['n_mean']     = df_['n_mean'].sum() 
    df_ICU['n_std']      = np.sqrt((df_['n_std']**2).sum()) 
    df_ICU['n_mean_ICU'] = df_['n_mean_ICU'].sum()
    df_ICU['n_std_ICU']  = np.sqrt((df_['n_std_ICU']**2).sum())

    df_ICU = df_ICU.set_index(['date'])


    if SUS:

        for j in range(n_samp_max):

            samp = np.random.uniform(size= int(df_ICU.loc[date]['n_mean_ICU']))  
            SUS_samp = samp <= p_SUS
            aux_.append(SUS_samp.sum())



        df_ICU.loc[date, 'n_mean_ICU_SUS'] = np.mean(aux_)
        df_ICU.loc[date, 'n_std_ICU_SUS']  = np.std(aux_)

        df_ICU['n_std_ICU_SUS'] = np.sqrt( df_ICU['n_std_ICU']**2 +  df_ICU['n_std_ICU_SUS']**2)

    return df_ICU

In [69]:
df1_

0.000    9.16  3.047503        0.00   
 05-09      1  3042098  0.065719     0.000    9.02  3.035081        0.00   
 10-14      2  2957679  0.063895     0.000    8.89  3.011409        0.00   
 15-19      3  3127121  0.067556     0.000   10.03  3.066453        0.00   
 20-24      4  3448513  0.074499     0.042   10.55  3.388558        0.42   
 25-29      5  3511090  0.075851     0.042   10.81  3.305016        0.36   
 30-34      6  3809963  0.082308     0.042   12.06  3.569851        0.51   
 35-39      7  3854180  0.083263     0.042   11.53  3.517503        0.41   
 40-44      8  3548754  0.076665     0.042   10.57  3.382232        0.41   
 45-49      9  3141667  0.067870     0.104    9.50  3.270622        0.95   
 50-54     10  2904703  0.062751     0.104    9.09  2.832335        0.81   
 55-59     11  2632224  0.056865     0.112    8.38  3.106786        0.97   
 60-64     12  2266765  0.048969     0.112    6.73  2.557836        0.66   
 65-69     13  1780635  0.038468     0.188    5.2

In [56]:
df1_ICU_ = daily_av(df1_, timeseries_data= df_I.index, SUS= True, p_SUS= p_SUS, n_samp_max= 100, n_samp_min= 100)

In [58]:
df1_ICU_

,n_mean,n_std,n_mean_ICU,n_std_ICU,n_mean_ICU_SUS,n_std_ICU_SUS
date,,,,,,
2020-04-06,1.0,0.967450,0.00,0.967450,0.00,0.967450
2020-04-07,1.0,0.968389,0.00,0.968389,0.00,0.968389
2020-04-08,1.0,0.968494,0.00,0.968494,0.00,0.968494
2020-04-09,2.0,1.375103,0.00,1.375103,0.00,1.375103
2020-04-10,4.0,1.948944,0.00,1.948944,0.00,1.948944
...,...,...,...,...,...,...
2020-06-06,224.0,14.208121,13.65,14.610880,8.30,14.733221
2020-06-07,228.0,14.151789,15.10,14.610754,9.74,14.738607
2020-06-08,226.0,14.600858,14.55,15.009176,8.58,15.096985


In [61]:
df1_ICU_.iloc[0]

n_mean            1.00000
n_std             0.96745
n_mean_ICU        0.00000
n_std_ICU         0.96745
n_mean_ICU_SUS    0.00000
n_std_ICU_SUS     0.96745
Name: 2020-04-06 00:00:00, dtype: float64

In [62]:
def correction(x, df_, T_ICU= 14):

    df_.iloc[x]['n_mean_ICU_cor'] = 0.
    df_.iloc[x]['n_std_ICU_cor'] = 0.
    
    if x <= T_ICU:
                   
        df_.iloc[x]['n_mean_ICU_cor'] = df_.iloc[x]['n_mean_ICU']
        df_.iloc[x]['n_std_ICU_cor']   = df_.iloc[x]['n_std_ICU']
             
    else:
        
        delta = df_.iloc[x]['n_mean_ICU'] - df_.iloc[x - T_ICU]['n_mean_ICU']
        
        df_.iloc[x]['n_mean_ICU_cor'] = np.heaviside(delta, 0) * delta
        df_.iloc[x]['n_std_ICU_cor']  = np.sqrt(df_.iloc[x]['n_std_ICU']**2 + df_.iloc[x - T_ICU]['n_std_ICU']**2)

In [65]:
len(df1_ICU_)

66

In [67]:
for j in range(len(df1_ICU_)):

    correction(j, df1_ICU_)


In [68]:
df1_ICU_

,n_mean,n_std,n_mean_ICU,n_std_ICU,n_mean_ICU_SUS,n_std_ICU_SUS
date,,,,,,
2020-04-06,1.0,0.967450,0.00,0.967450,0.00,0.967450
2020-04-07,1.0,0.968389,0.00,0.968389,0.00,0.968389
2020-04-08,1.0,0.968494,0.00,0.968494,0.00,0.968494
2020-04-09,2.0,1.375103,0.00,1.375103,0.00,1.375103
2020-04-10,4.0,1.948944,0.00,1.948944,0.00,1.948944
...,...,...,...,...,...,...
2020-06-06,224.0,14.208121,13.65,14.610880,8.30,14.733221
2020-06-07,228.0,14.151789,15.10,14.610754,9.74,14.738607
2020-06-08,226.0,14.600858,14.55,15.009176,8.58,15.096985


In [20]:
df1_ICU

,n_mean,n_std,n_mean_ICU,n_std_ICU,n_mean_ICU_SUS,n_std_ICU_SUS
date,,,,,,
2020-04-06,1.0,0.968076,0.00,0.968076,0.00,0.968076
2020-04-07,1.0,0.965150,0.00,0.965150,0.00,0.965150
2020-04-08,1.0,0.963684,0.00,0.963684,0.00,0.963684
2020-04-09,2.0,1.368255,0.00,1.368255,0.00,1.368255
2020-04-10,4.0,1.922225,0.00,1.922225,0.00,1.922225
...,...,...,...,...,...,...
2020-06-06,224.0,14.812778,9.35,17.976001,6.02,18.029315
2020-06-07,228.0,15.176863,9.02,18.321058,5.52,18.374732
2020-06-08,226.0,14.371928,8.27,17.712190,4.89,17.765685


## Private ICU beds

In [21]:
df1_ICU['n_mean_ICU_PRIVATE'] = df1_ICU['n_mean_ICU'] - df1_ICU['n_mean_ICU_SUS'] 
df1_ICU['n_std_ICU_PRIVATE'] = np.sqrt(  df1_ICU['n_std_ICU']**2 + df1_ICU['n_std_ICU_SUS']  )

## Saving the results. 

In [22]:
file1 = 'results/dfs/df_ICU_' + state_or_city + '_' + name_file + '_fit_until_' + fit_until + '.pkl'
df1_ICU.to_pickle(file1) 